# Ensemble Learning

## Initial Imports

In [75]:
import warnings
warnings.filterwarnings('ignore')

In [76]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [77]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [78]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
loan_stats = pd.read_csv(file_path)

# Preview the data
loan_stats.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [85]:
loan_stats_encoded = pd.get_dummies(loan_stats, columns=["home_ownership", "verification_status", "issue_d", "pymnt_plan",
                                                        "initial_list_status", "next_pymnt_d", "application_type",
                                                        "hardship_flag", "debt_settlement_flag"])

# Create our features
X = loan_stats_encoded.copy()
X.drop("loan_status", axis=1, inplace=True)
X.head()


# Create our target
y = loan_stats_encoded["loan_status"]

In [86]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [87]:
# Check the balance of our target values
# YOUR CODE HERE
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [88]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=55)
X_train.shape

(51612, 95)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [89]:
# Create the StandardScaler instance
# YOUR CODE HERE
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [90]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)

In [91]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [107]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from imblearn.ensemble import BalancedRandomForestClassifier
brfc=BalancedRandomForestClassifier(n_estimators=100, random_state=55)
brfc = brfc.fit(X_train_scaled, y_train)
predictions = brfc.predict(X_test_scaled)

In [108]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, predictions)

0.7858784473953013

In [109]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[   61,    28],
       [ 1945, 15171]])

In [110]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.69      0.89      0.06      0.78      0.60        89
   low_risk       1.00      0.89      0.69      0.94      0.78      0.62     17116

avg / total       0.99      0.89      0.69      0.93      0.78      0.62     17205



In [111]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
feature_importance = brfc.feature_importances_
sorted(zip(feature_importance, X.columns), reverse=True)

[(0.0744022921579115, 'total_rec_prncp'),
 (0.06650865121544425, 'total_pymnt_inv'),
 (0.06198439182404872, 'total_rec_int'),
 (0.060926847536801124, 'last_pymnt_amnt'),
 (0.06092360426495602, 'total_pymnt'),
 (0.03514237036103959, 'int_rate'),
 (0.023648445890193447, 'issue_d_Jan-2019'),
 (0.0185737128270495, 'max_bal_bc'),
 (0.01735661218335714, 'dti'),
 (0.016926376649039437, 'installment'),
 (0.016192838646386425, 'annual_inc'),
 (0.014907940489881734, 'out_prncp'),
 (0.014534705875088537, 'mo_sin_old_il_acct'),
 (0.014194738661163418, 'bc_util'),
 (0.013990363479251672, 'mths_since_rcnt_il'),
 (0.013769240439860761, 'loan_amnt'),
 (0.01356243679832245, 'revol_bal'),
 (0.013381530070011659, 'total_bc_limit'),
 (0.012927951437782342, 'mo_sin_old_rev_tl_op'),
 (0.012910797718278246, 'mths_since_recent_bc'),
 (0.01280033063361172, 'issue_d_Mar-2019'),
 (0.012675306726859391, 'total_bal_ex_mort'),
 (0.012484866734576168, 'out_prncp_inv'),
 (0.01244865836416633, 'mths_since_recent_inq')

### Easy Ensemble Classifier

In [112]:
# Train the Classifier
# YOUR CODE HERE
from imblearn.ensemble import EasyEnsembleClassifier 
easy_class=EasyEnsembleClassifier(n_estimators=100, random_state=55)
easy_class = easy_class.fit(X_train_scaled, y_train)
predictions = easy_class.predict(X_test_scaled)

In [113]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
balanced_accuracy_score(y_test, predictions)

0.8954368867030258

In [114]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, predictions)

array([[   75,    14],
       [  887, 16229]])

In [115]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.84      0.95      0.14      0.89      0.79        89
   low_risk       1.00      0.95      0.84      0.97      0.89      0.81     17116

avg / total       0.99      0.95      0.84      0.97      0.89      0.81     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    Easy Ensemble.

2. Which model had the best recall score?

    Easy Ensemble.

3. Which model had the best geometric mean score?

    Easy Ensemble.

4. What are the top three features?

   * (0.0744022921579115, 'total_rec_prncp')
   * (0.06650865121544425, 'total_pymnt_inv')
   * (0.06198439182404872, 'total_rec_int')